In [ ]:
import pandas as pd
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
import numpy_financial as npf

def get_closest_price(date, market):
    market_value = None
    round = 0
    while round < 7:
        round = round + 1
        idx = int(date.strftime('%Y%m%d'))
        if idx in market.index:
            market_value = merged_data.loc[int(date.strftime('%Y%m%d'))]
            break
        else:
            date = date + timedelta(days=1)
            
    return market_value

def rebalance_budge(budge):
    comp = [0.5, 0.5]
    return [budge * c for c in comp]

# 读取股票和债券的价格数据
# stock_data = pd.read_excel('C:\\Users\\wayne\\Downloads\\000300perf.xlsx')
stock_data = pd.read_excel('C:\\Users\\wayne\\Downloads\\000905perf.xlsx')
bond_data = pd.read_excel('C:\\Users\\wayne\\Downloads\\H11006perf.xlsx')

# 合并两个数据集，按照日期Date对齐
merged_data = pd.merge(stock_data, bond_data, on='日期Date', how='inner', suffixes=('_stock', '_bond'))
merged_data.set_index('日期Date', inplace=True)
# 初始化投资金额
portfolio = 10000
cash_amount = portfolio
stock_share = 0
bond_share = 0

# 模拟投资策略
next_rebalance_date = pd.to_datetime('2013-01-01')
end_date = pd.to_datetime('2023-12-31')

result_data = []
while True:
    market_value = get_closest_price(next_rebalance_date, merged_data)
    if market_value is None:
        break
    # print(next_rebalance_date, market_value.name, market_value['收盘Close_stock'], market_value['收盘Close_bond'])
    stock_price = market_value['收盘Close_stock']
    bond_price = market_value['收盘Close_bond']
    
    # 相当于卖出换成现金
    stock_amt = stock_share * stock_price
    bond_amt = bond_share * bond_price
    total_amt = cash_amount + stock_amt + bond_amt

    total_rate = (total_amt - portfolio) * 100 / portfolio
    stock_rate = (stock_amt - portfolio / 2) * 100 / portfolio / 2
    bond_rate = (bond_amt - portfolio / 2) * 100 / portfolio / 2
    # 再平衡购买 
    stock_budge, bond_budge = rebalance_budge(total_amt)
    stock_share = stock_budge / stock_price
    bond_share = bond_budge / bond_price
    cash_amount = 0

    # set next rebalance date
    next_rebalance_date = next_rebalance_date + relativedelta(months=12)
    result_data.append({
        'date': market_value.name,
        'total': total_amt,
        'total_rate': total_rate,
        'stock_amt': stock_amt,
        'stock_rate': stock_rate,
        'stock_price': stock_price,
        'bond_amt': bond_amt,
        'bond_rate': bond_rate,
        'bond_price': bond_price,
    })
    
x = [-10000.000000,0,0,0,0,0,0,0,0,0,16304.922720,]
npf.irr(x)

result = pd.json_normalize(result_data)

result.head(20)